# Simpel XML parser from Buienradar

### packages used:

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import folium
from folium.features import DivIcon


### Creating two functions, one to get the xml data from the website and second to extract data

In [59]:
def download(url):
    try:
        html_page = requests.get(url).text
        soup = BeautifulSoup(html_page,'xml')
        return(soup)
    except:
        print('failed to get webpage')
        return(None)

def get_station_info(url):
    #get all the tags from the xml file
    tag_list = [tag.name for tag in soup_page.find('weerstation').find_all()]
    #list to store station data
    weerstation = []
    for i in tag_list:
        weerstation_info_row = [i.text for i in soup_page.findAll(i)]
        weerstation.append(weerstation_info_row)
    return (weerstation, tag_list)



### Collect data and create dataframe for checking

In [67]:
url = 'https://data.buienradar.nl/1.0/feed/xml'
weerstation_taglist = get_station_info(url)

df = pd.DataFrame(weerstation_taglist[0],index = weerstation_taglist[1]).T
df = df.dropna()
df.head()


,stationcode,stationnaam,lat,lon,datum,luchtvochtigheid,temperatuurGC,windsnelheidMS,windsnelheidBF,windrichtingGR,...,luchtdruk,zichtmeters,windstotenMS,regenMMPU,zonintensiteitWM2,icoonactueel,temperatuur10cm,url,latGraden,lonGraden
0,6391,Meetstation Arcen,51.50,6.20,07/01/2020 11:40:00,96,18.1,2.5,2,217,...,-,-,7.5,0.5,170,https://www.buienradar.nl/resources/images/ico...,18.4,http://www.buienradar.nl/resources/images/logo...,51.83,6.33
1,6275,Meetstation Arnhem,52.07,5.88,07/01/2020 11:40:00,99,17.9,4.9,3,223,...,1006.2,6290,7.2,0.2,303,https://www.buienradar.nl/resources/images/ico...,17.8,https://www.buienradar.nl/nederland/weerberich...,52.11,6.47
2,6249,Meetstation Berkhout,52.65,4.98,07/01/2020 11:40:00,81,17.9,7.7,4,249,...,-,28200,11.8,-,244,https://www.buienradar.nl/resources/images/ico...,18.0,https://www.buienradar.nl/nederland/weerberich...,53.08,5.64
3,6308,Meetstation Cadzand,51.38,3.38,07/01/2020 11:40:00,-,-,6.6,4,217,...,-,-,9.4,-,-,https://www.buienradar.nl/resources/images/ico...,-,https://www.buienradar.nl/nederland/weerberich...,51.64,3.64
4,6260,Meetstation De Bilt,52.10,5.18,07/01/2020 11:40:00,80,18.6,4.5,3,220,...,1005.1,24900,8.7,-,437,https://www.buienradar.nl/resources/images/ico...,19.9,https://www.buienradar.nl/nederland/weerberich...,52.17,5.31


### Create folium map to visualize the data

In [65]:
longitude = 5.203125
latitude = 51.995503

# create map of Delft using latitude and longitude values
map_netherlands = folium.Map(location=[latitude, longitude], zoom_start=8,tiles='OpenStreetMap')

#add markers to map
for lat, lng, temp, stationnaam in zip(df['lat'], df['lon'], df['temperatuurGC'],df['stationnaam']):
    label = '{} temperatuur: {}'.format(stationnaam, temp)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#4868FF',
        fill_opacity=0.7,
        parse_html=False).add_to(map_netherlands)      

    folium.map.Marker(
        [lat,lng],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 15pt">%s</div>' % temp
            )
    ).add_to(map_netherlands)  
map_netherlands